GAN で PyTorch実装を学ぶ  
- Reference
  - https://github.com/eriklindernoren/PyTorch-GAN
  - https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/gan/gan.py
  

In [1]:
import os
import numpy as np

In [2]:
import torchvision.transforms as transforms
from torchvision.utils import save_image

In [3]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

In [4]:
import torch.nn as nn
#import torch.nn.functional as F
import torch

In [5]:
torch.__version__

'1.6.0+cu101'

In [6]:
DATA_DIR_PATH = '/content/drive/My Drive/project/ML/pytorch-gan/data'

OUTPUT_DIR_PATH = os.path.join(DATA_DIR_PATH, '01_out')

In [7]:
os.makedirs(os.path.join(OUTPUT_DIR_PATH, 'images'), exist_ok=True)
#os.makedirs(os.path.join(OUTPUT_DIR_PATH, 'saved_models'), exist_ok=True)

In [26]:
channels = 1
img_size = 28
img_shape = (channels, img_size, img_size)
latent_dim = 100

cuda = True if torch.cuda.is_available() else False
print(cuda)

True


In [27]:
class Generator(nn.Module):
    def __init__(self, img_shape, latent_dim):
        super(Generator, self).__init__()

        self.img_shape = img_shape
        self.latent_dim = latent_dim

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]

            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))

            layers.append(nn.LeakyReLU(0.2, inplace=True))

            return layers

        self.model = nn.Sequential(
            *block(self.latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *self.img_shape)
        return img

In [28]:
class Discriminator(nn.Module):
    def __init__(self, img_shape):
        super(Discriminator, self).__init__()

        self.img_shape = img_shape

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(self.img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

In [29]:
# Loss function
adversarial_loss = torch.nn.BCELoss()
# Use BCEWithLogitsLoss?

In [30]:
# Initialize generator and discriminator
generator = Generator(img_shape, latent_dim)
discriminator = Discriminator(img_shape)

In [31]:
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [32]:
# Configure data loader
MNIST_DATA_DIR_PATH = os.path.join(DATA_DIR_PATH, 'mnist')

batch_size = 64

#os.makedirs("../../data/mnist", exist_ok=True)
os.makedirs(MNIST_DATA_DIR_PATH, exist_ok=True)

dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        MNIST_DATA_DIR_PATH,
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

In [33]:
lr = 0.0002
b1 = 0.5
b2 = 0.999

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

### Training

In [34]:
n_epochs = 1
sample_interval = 400

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

for epoch in range(n_epochs):

    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            file_name = os.path.join(OUTPUT_DIR_PATH, 'images', '%d.png' % batches_done)
            save_image(gen_imgs.data[:25], file_name, nrow=5, normalize=True)

[Epoch 0/1] [Batch 0/938] [D loss: 0.717321] [G loss: 0.683718]
[Epoch 0/1] [Batch 1/938] [D loss: 0.629620] [G loss: 0.681075]
[Epoch 0/1] [Batch 2/938] [D loss: 0.564453] [G loss: 0.678694]
[Epoch 0/1] [Batch 3/938] [D loss: 0.507961] [G loss: 0.675966]
[Epoch 0/1] [Batch 4/938] [D loss: 0.465221] [G loss: 0.672695]
[Epoch 0/1] [Batch 5/938] [D loss: 0.435114] [G loss: 0.667985]
[Epoch 0/1] [Batch 6/938] [D loss: 0.412640] [G loss: 0.662220]
[Epoch 0/1] [Batch 7/938] [D loss: 0.402126] [G loss: 0.655870]
[Epoch 0/1] [Batch 8/938] [D loss: 0.396555] [G loss: 0.647792]
[Epoch 0/1] [Batch 9/938] [D loss: 0.399529] [G loss: 0.635446]
[Epoch 0/1] [Batch 10/938] [D loss: 0.398387] [G loss: 0.626196]
[Epoch 0/1] [Batch 11/938] [D loss: 0.401212] [G loss: 0.615050]
[Epoch 0/1] [Batch 12/938] [D loss: 0.406717] [G loss: 0.604823]
[Epoch 0/1] [Batch 13/938] [D loss: 0.413522] [G loss: 0.593155]
[Epoch 0/1] [Batch 14/938] [D loss: 0.417937] [G loss: 0.585852]
[Epoch 0/1] [Batch 15/938] [D loss: